In [ ]:
!pip install onnx
!pip install onnxscript

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model = model.to(torch.float16)
model.eval()

In [11]:
import cv2
import torchvision.transforms as transforms 
from PIL import Image

input_image = Image.open("1.jpg")


preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [15]:
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to(torch.float16).to('cuda')
    model.to('cuda')
with torch.no_grad():
    output = model(input_batch)

In [18]:
torch.onnx.export(model,         # model being run 
         input_batch,       # model input (or a tuple for multiple inputs) 
         "Network.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=11,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['input'],   # the model's input names 
         output_names = ['output'], # the model's output names 
         dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes 
                                'output' : {0 : 'batch_size'}})
print(" ") 
print('Model has been converted to ONNX') 

 
Model has been converted to ONNX


In [21]:
!pip install onnxruntime



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import onnx
onnx_model = onnx.load("Network.onnx")
onnx.checker.check_model(onnx_model)

import onnxruntime as ort
import numpy as np

ort_sess = ort.InferenceSession('Network.onnx')
outputs = ort_sess.run(None, {'input': input_batch.cpu().numpy()})

outputs

In [28]:
ort_sess = ort.InferenceSession('Network.onnx')
data = {'input': input_batch.cpu().numpy()}

from viztracer import VizTracer

with VizTracer(output_file="onnx_inference.json") as tracer:
    # Something happens here
    outputs = ort_sess.run(None, data)

Total Entries: 7                                                                
Use the following command to open the report:
vizviewer C:\workspace\projects\talk\onnx_inference.json


In [31]:
input_batch = input_batch.to(torch.float16).to('cpu')
model.to('cpu')

with VizTracer(output_file="torch_inference.json") as tracer:
    with torch.no_grad():
        output = model(input_batch)

Total Entries: 1947                                                             
Use the following command to open the report:
vizviewer C:\workspace\projects\talk\torch_inference.json


In [32]:
print(input_batch.shape)

torch.Size([1, 3, 224, 224])
